In [1]:
import duckdb

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from logic import transformations, files
import datetime as dt
from pathlib import Path
import xarray as xr
from adlfs import AzureBlobFileSystem
from azure.identity import DefaultAzureCredential
from azure.storage.filedatalake import DataLakeServiceClient
import zarr
import pandas as pd
import numpy as np
import os
import sparse

In [12]:
client = DataLakeServiceClient(
    account_url="{}://{}.dfs.core.windows.net".format("https", "saenigmaarchiveprod"),
    credential=DefaultAzureCredential(),
)

In [25]:
fs_client = client.get_file_system_client("uniform")

In [26]:
file_client = fs_client.get_file_client("forecast_nwp_dmi_harmonie_dini_sf/schema_version=6f2c8ca4e0b854b8ab4f5691ff143357/year=2024/month=09/day=07/hour=20/2024-09-07T20-02-02.988228_b3c39ba5-776b-4b71-9b1e-f692b70a8c5a.parquet")

In [27]:
with open("/home/uch/PVForecast/data/.datalake/forecast_nwp_dmi_harmonie_dini_sf.parquet", "wb") as f:
    content = file_client.download_file().readall()
    f.write(content)

In [15]:
import pandas as pd

In [28]:
df = pd.read_parquet("/home/uch/PVForecast/data/.datalake/forecast_nwp_dmi_harmonie_dini_sf.parquet")

In [29]:
df

,latitude,longitude,time_utc,created_time_utc,forecast_value,parameter,altitude_m
0,54.300426,7.007448,2024-09-09 16:00:00+00:00,2024-09-07 15:00:00+00:00,2.903856e+02,temperature,2
1,54.317991,7.014052,2024-09-09 16:00:00+00:00,2024-09-07 15:00:00+00:00,2.904041e+02,temperature,2
2,54.314134,7.044161,2024-09-09 16:00:00+00:00,2024-09-07 15:00:00+00:00,2.904090e+02,temperature,2
3,54.310269,7.074265,2024-09-09 16:00:00+00:00,2024-09-07 15:00:00+00:00,2.904236e+02,temperature,2
4,54.306397,7.104364,2024-09-09 16:00:00+00:00,2024-09-07 15:00:00+00:00,2.904461e+02,temperature,2
...,...,...,...,...,...,...,...
327987,55.365121,15.389465,2024-09-09 16:00:00+00:00,2024-09-07 15:00:00+00:00,2.011735e+07,accumulated_direct_radiation,0
327988,55.399885,15.310227,2024-09-09 16:00:00+00:00,2024-09-07 15:00:00+00:00,2.016650e+07,accumulated_direct_radiation,0
327989,55.393963,15.340132,2024-09-09 16:00:00+00:00,2024-09-07 15:00:00+00:00,2.026071e+07,accumulated_direct_radiation,0
327990,55.388034,15.370029,2024-09-09 16:00:00+00:00,2024-09-07 15:00:00+00:00,2.023511e+07,accumulated_direct_radiation,0


In [64]:
with open("ds1.nc", "wb") as f:
    file_client = client.get_file_client("analysis", "uch/netcdf/date=2024-06-08/20240608T000000Z_20240606T180000Z.nc")
    f.write(file_client.download_file().readall())

In [3]:
ds1 = xr.open_dataset("ds1.nc")

In [21]:
dst = ds1.where(1==0)
dst = xr.zeros_like(ds1)

In [28]:
dst

<xarray.Dataset> Size: 110MB
Dimensions:                            (y: 1606, x: 1906)
Coordinates:
    latitude                           (y, x) float64 24MB ...
    longitude                          (y, x) float64 24MB ...
    number                             int32 4B ...
    calculation_time_utc               datetime64[ns] 8B ...
    step                               timedelta64[ns] 8B ...
    time_utc                           datetime64[ns] 8B ...
Dimensions without coordinates: y, x
Data variables:
    temperature_K                      (y, x) float32 12MB 0.0 0.0 ... 0.0 0.0
    wind_u_m_s                         (y, x) float32 12MB 0.0 0.0 ... 0.0 0.0
    wind_v_m_s                         (y, x) float32 12MB 0.0 0.0 ... 0.0 0.0
    accumulated_direct_radiation_W_m2  (y, x) float32 12MB 0.0 0.0 ... 0.0 0.0
    accumulated_global_radiation_W_m2  (y, x) float32 12MB 0.0 0.0 ... 0.0 0.0
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ekmi
    GRIB_centreDescription:  Copenhagen
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Copenhagen
    history:                 2024-08-08T13:31 GRIB to CDM+CF via cfgrib-0.9.1...
    parameter:               temperature

In [66]:
ds1 = ds1.expand_dims(dim=["time_utc", "calculation_time_utc"])

In [16]:
ds2 = xr.open_dataset("ds2.nc")

In [42]:
ds2 = ds2.expand_dims(dim=["time_utc", "calculation_time_utc"])

ValueError: Dimension time_utc already exists.

In [40]:
ds1 = 0

In [67]:
ds1.to_zarr(store="test_store.zarr", compute=False)

Delayed('_finalize_store-284e9fa6-732c-4b4b-93ac-9dc7d8fa8d62')

In [35]:
(
    ds1
    .drop_vars(['latitude', 'longitude', 'number', 'step'])
    .to_zarr(store="test_store.zarr", region={"time_utc": slice(0,1), "calculation_time_utc": slice(0,1)})
)

In [38]:
(
    ds2
    .drop_vars(['number', 'step', 'latitude', 'longitude'])
    .to_zarr(store="test_store.zarr", region={"time_utc": slice(1,2), "calculation_time_utc": slice(1,2)})
)

ValueError: variable 'accumulated_direct_radiation_W_m2' already exists with different dimension sizes: {'time_utc': 0, 'calculation_time_utc': 0, 'y': 1606, 'x': 1906} != {'time_utc': 1, 'calculation_time_utc': 1, 'y': 1606, 'x': 1906}. to_zarr() only supports changing dimension sizes when explicitly appending, but append_dim=None. If you are attempting to write to a subset of the existing store without changing dimension sizes, consider using the region argument in to_zarr().

In [ ]:
dsz = xr.open_dataset("test_store.zarr", engine="zarr")

In [ ]:
dsz

In [ ]:
fs = AzureBlobFileSystem(account_name="saenigmaarchivedev", credential=DefaultAzureCredential())

In [ ]:
store = fs.get_mapper("az://analysis/uch/data/zarr_store")

In [ ]:
ds = xr.open_zarr(store, consolidated=True)

In [ ]:
ds

In [ ]:
files.nwp.zarr_to_dataset(
    source=Path("az://analysis/uch/data/zarr"),
    start_time=dt.datetime(2024, 6, 1),
    end_time=dt.datetime(2024,6,1),
    storage_options=storage_options
)

In [ ]:
from azure.storage.filedatalake import DataLakeServiceClient
from azure.identity import DefaultAzureCredential
from logic import transformations, files
from io import BytesIO
from adlfs import AzureBlobFileSystem
import fsspec
import xarray as xr

In [ ]:
source = AzureBlobFileSystem(account_name="saenigmaarchiveprod", credential=DefaultAzureCredential())

In [ ]:
source.get_file("landing/forecast_nwp_dmi_harmonie_dini_sf/schema_version=None/year=2024/month=06/day=01/hour=00/2024-06-01T00-45-50.191399_d635c80e-d713-4834-af99-34ea51c36f69.grib", "local.grib")

In [ ]:
mapper = source.get_mapper("landing/forecast_nwp_dmi_harmonie_dini_sf/schema_version=None/year=2024/month=06/day=01/hour=00", check=True)

In [ ]:
mapper.get("2024-06-01T00-45-50.191399_d635c80e-d713-4834-af99-34ea51c36f69.grib")

In [ ]:
xr.open_dataset(
        mapper,
        engine="zarr",
        backend_kwargs={
            "filter_by_keys": {
                "stepType": "instant",
                "typeOfLevel": "heightAboveGround",
                "parameterCategory": 0,
                "parameterNumber": 0,
                "level": 2
            },
            "indexpath": "",
        },
        storage_options=dict()
    )

In [ ]:
fs = fsspec.AbstractFileSystem(storage_options=storage_options)

In [ ]:
fs.find("az://landing/forecast_nwp_dmi_harmonie_dini_sf/schema_version=None/year=2024/month=06")

In [ ]:
class Archive:

    def __init__(self):
        datalake_service_client = DataLakeServiceClient(
            account_url="https://saenigmaarchiveprod.dfs.core.windows.net",
            credential=DefaultAzureCredential(),
        )
        self.file_system_client = datalake_service_client.get_file_system_client("landing")


    def get_files(self):
        files = (self.file_system_client.get_paths("forecast_nwp_dmi_harmonie_dini_sf/schema_version=None/year=2024/month=06"))
        return [file["name"] for file in files]

In [ ]:
archive = Archive()

In [ ]:
archive.get_files()

In [ ]:
sandbox_datalake_service_client = DataLakeServiceClient(
    account_url="https://saenigmaarchivedev.dfs.core.windows.net",
    credential=DefaultAzureCredential(),
)

In [ ]:
archive_datalake_service_client = DataLakeServiceClient(
    account_url="https://saenigmaarchiveprod.dfs.core.windows.net",
    credential=DefaultAzureCredential(),
)

In [ ]:
file_system_client = archive_datalake_service_client.get_file_system_client("landing")

In [ ]:
file_list = file_system_client.get_paths("forecast_nwp_dmi_harmonie_dini_sf/schema_version=None/year=2024/month=06/day=01/hour=12")

In [ ]:
file_list = list(file_list)

In [ ]:
file_client = file_system_client.get_file_client(file_list[1])

In [ ]:
content = file_client.download_file()

In [ ]:
with open("test.grib", "wb") as f:
    content.readinto(f)

In [ ]:
ds = transformations.nwp.grib_to_dataset("test.grib")

In [ ]:
destination = sandbox_datalake_service_client.get_directory_client(file_system="analysis", directory="uch/netcdf")

In [ ]:
files.nwp.dataset_to_netcdf(ds, destination)